In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：评估AutoML视频分类模型的批量预测结果

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_video_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_video_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/automl_video_classification_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概览

本笔记本演示了如何使用Vertex AI分类模型评估组件来评估AutoML视频分类模型。模型评估帮助您根据评估指标确定模型的性能，并在必要时改进模型。

了解更多关于[Vertex AI模型评估](https://cloud.google.com/vertex-ai/docs/evaluation/introduction)和[视频数据分类](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_videos)。

### 目标

在本教程中，您将学习如何使用Vertex AI AutoML Video分类模型进行训练，并通过使用`google_cloud_pipeline_components`执行Vertex AI管道作业来评估它：

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI `数据集`
- Vertex AI `训练`(AutoML Video分类)
- Vertex AI `模型注册表`
- Vertex AI `管道`
- Vertex AI `批量预测`

执行的步骤包括：

- 创建一个`Vertex AI数据集`。
- 在`Vertex AI数据集`资源上训练一个AutoML Video分类模型。
- 将训练好的`AutoML Vertex AI模型资源`导入到管道中。
- 在管道内运行一个批量预测作业。
- 使用分类评估组件评估AutoML模型。
- 将分类指标导入AutoML Vertex AI模型资源中。

### 数据集

本教程使用的数据集是来自麻省理工学院的[人体运动数据集](http://cbcl.mit.edu/publications/ps/Kuehne_etal_iccv11.pdf)中的高尔夫挥杆识别部分。在本教程中使用的数据集版本存储在公共云存储桶中。训练好的模型预测高尔夫挥杆开始的起始帧。

成本

本教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装执行此笔记所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-pipeline-components==1.0.26 \
                                 google-cloud-storage

只有协作：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 请查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的“REGION”变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"
DATA_REGION = "US"

### 认证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动认证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 由于您已经通过认证，因此无需进行任何操作。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在以下网址为您的服务账户授予Cloud Storage权限：https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

#### 服务账户

您可以使用服务账户创建 Vertex AI 管道作业。如果您不想使用项目的计算引擎服务账户，请将`SERVICE_ACCOUNT`设置为另一个服务账户ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置Vertex AI Pipelines的服务账户访问权限
运行以下命令，将您的服务账户访问权限授予您在上一步创建的存储桶中读取和写入管道工件的权限。您只需要针对每个服务账户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

In [ ]:
import json

import google.cloud.aiplatform as aiplatform
import matplotlib.pyplot as plt
from google.cloud import aiplatform_v1, storage

初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 训练数据的位置

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/video/automl_classification/hmdb_split_40_mp4_step2.csv"
)

### 快速查看您的数据

本教程使用存储在公共云存储桶中的MIT人体运动数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中行数（`wc -l`）来计算示例数量，然后查看前几行。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法创建`Vertex AI Dataset`资源，该方法需要以下参数：

- `display_name`: `Vertex AI Dataset`资源的可读名称。
- `gcs_source`: 一个或多个数据集索引文件列表，用于将数据项导入`Vertex AI Dataset`资源。

该操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="MIT Human Motion",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

### 创建和运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：

1. 创建一个训练流水线。
2. 运行这个流水线。

#### 创建训练流水线

使用`AutoMLVideoTrainingJob`类创建一个AutoML训练流水线，包括以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：为模型训练指定的任务类型。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标跟踪模型。
  - `action_recognition`：视频动作识别模型。

In [ ]:
training_job = aiplatform.AutoMLVideoTrainingJob(
    display_name="hmdb",
    prediction_type="classification",
)

print(training_job)

#### 运行训练流程

接下来，您可以通过调用`run`方法来运行作业以启动训练任务，使用以下参数：

- `dataset`：要训练模型的`Vertex AI Dataset`资源。
- `model_display_name`：训练模型的人类可读的名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。

完成`run`方法后会返回`Model`资源。

训练流程的执行可能需要超过24小时才能完成。

In [ ]:
import os

if os.getenv("IS_TESTING"):
    sys.exit(0)

In [ ]:
model = training_job.run(
    dataset=dataset,
    model_display_name="hmdb",
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

print(model)

## 列出训练的模型评估
在您的模型训练完成后，您可以查看其评估分数。

您可以使用 Vertex AI 模型资源的 `get_model_evaluation` 方法来查看模型的评估结果。

与 Vertex AI 数据集一样，您可以使用训练模型时创建的模型变量的引用，也可以根据下面给定的模型显示名称从项目中的所有模型列表中筛选。

In [ ]:
# Get Vertex AI Model resource ID using the display_name
models = aiplatform.Model.list(filter="display_name=hmdb")

if len(models) != 0:
    # Get the model object
    MODEL_RSC_NAME = models[0].resource_name
    print("Vertex AI Model resource name:", MODEL_RSC_NAME)
    model = aiplatform.Model(MODEL_RSC_NAME)

    # Print the evaluation metrics
    model_eval = model.get_model_evaluation()
    evaluation = model_eval.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

获取测试项

在管道中，您需要一些数据样本来创建批量预测任务。因此，您可以从数据集中使用一些任意示例作为测试项。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

if len(test_items[0]) == 5:
    _, test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    _, test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项目
对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_URI/$file_1
! gsutil cp $test_item_2 $BUCKET_URI/$file_2

test_item_1 = BUCKET_URI + "/" + file_1
test_item_2 = BUCKET_URI + "/" + file_2

### 制作Pipeline输入文件

现在，制作一个用于评估Pipeline的输入文件，并将其存储在云存储存储桶中。本教程的输入文件以JSONL格式存储。在JSONL文件中，您为每个视频文件的每一行创建一个字典条目。该字典包含以下键值对：

- `content`：视频在云存储中的路径。
- `mimeType`：内容类型。在我们的示例中，它是一个`avi`文件。
- `timeSegmentStart`：视频中用于预测的起始时间戳。*注意*，时间戳必须以字符串形式指定，并后跟s（秒）、m（分钟）或h（小时）。
- `timeSegmentEnd`：视频中用于预测的结束时间戳。
- `outputLabel`：批量预测标签。

In [ ]:
test_filename = "ground_truth.jsonl"
gcs_ground_truth_uri = BUCKET_URI + "/" + test_filename

data_1 = {
    "content": test_item_1,
    "mimeType": "video/mp4",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
    "outputLabel": test_label_1,
}
data_2 = {
    "content": test_item_2,
    "mimeType": "video/mp4",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
    "outputLabel": test_label_2,
}


bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI[5:])
blob = bucket.blob(blob_name=test_filename)
data = json.dumps(data_1) + "\n" + json.dumps(data_2) + "\n"
blob.upload_from_string(data)
print(gcs_ground_truth_uri)

检查`ground_truth.jsonl`文件的内容。

In [ ]:
! gsutil cat $gcs_ground_truth_uri

运行用于模型评估的流水线

现在，您可以运行一个 Vertex AI 批处理预测作业，并使用流水线生成其结果的评估和特征归因。

为此，您可以调用 `evaluate` 函数创建一个 Vertex AI 流水线。了解更多关于 [evaluate 函数](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/models.py#L5127)。

### 定义运行评估函数的参数

指定运行`evaluate`函数所需的参数。

以下是`evaluate`函数参数的说明：

- `prediction_type`：评估运行所涉及的问题类型。'classification'和'regression'是目前支持的问题类型。
- `target_field_name`：要用作分类目标的列的名称。
- `gcs_source_uris`：批量预测输入实例的Cloud Storage桶URI列表。
- `class_labels`：目标列中的类标签列表。
- `generate_feature_attributions`：可选项。模型评估作业是否应生成特征归因。如果未指定，默认为False。

In [ ]:
LABEL_COLUMN = "outputLabel"
CLASS_LABELS = ["brush_hair", "cartwheel"]

job = model.evaluate(
    prediction_type="classification",
    target_field_name=LABEL_COLUMN,
    gcs_source_uris=[gcs_ground_truth_uri],
    class_labels=CLASS_LABELS,
    generate_feature_attributions=False,
)

print("Waiting model evaluation is in process")
job.wait()

获取模型评估结果
在评估管道完成后，运行下面的单元格打印评估指标。

In [ ]:
model_evaluation = job.get_model_evaluation()

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[0]
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

## 可视化指标
使用条形图可视化可用的指标，例如auRoc和logLoss。

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(5, 3))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 模型
- AutoML训练作业
- Cloud Storage存储桶

In [ ]:
# If the bucket needs to be deleted too, please set "delete_bucket" to True
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the training job
training_job.delete()

# Delete the evaluation pipeline
job.delete()

# Delete the Cloud storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI